# Installs and Imports

In [1]:
!pip install transformers --quiet
!pip install -U datasets transformers ninja -q
!pip install rouge_score
!pip install -U datasets transformers ninja -q
!pip install evaluate



import pprint
import timeit
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import os
import zipfile
from io import BytesIO, StringIO
import pprint
import operator
from pprint import pprint
#from fast_ml.model_development import train_valid_test_split 
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
import tensorflow as tf


SHUFFLE_BUFFER = 500
BATCH_SIZE = 2

from transformers import pipeline
import torch
#from cleantext import clean
from pathlib import Path
from transformers import AutoTokenizer
from datasets import load_dataset, load_metric
from transformers import AutoTokenizer
from transformers import AutoModelForSeq2SeqLM
import torch
from transformers import pipeline
from transformers import LEDForConditionalGeneration, LEDTokenizer



from google.colab import drive
drive.mount('/content/drive')
%cd drive/MyDrive/Colab Notebooks/Booksum

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab Notebooks/Booksum


# Load Data, Remove Columns, and Filter Nulls

In [2]:
dataset = load_dataset("csv", data_files="Wikipedia_movie_meta_data.csv",split="train")
dataset_val=dataset.train_test_split(test_size=0.2)
dataset_train = dataset_val['train']
dataset_val = dataset_val['test']

dataset_train = dataset_train.remove_columns(['Age Restrict', 'Akas', 'Awards', 'Budget', 'Cast', 'Casting Directors', 'Countries', 'Directors', 'Filename', 'Genres', 'Imdb Id', 'Keywords', 'Opening Weekend', 'Plot', 'Producers', 'Production Companies','Script Department', 'Synopsis', 'Taglines', 'Writers', 'Year', 'F1', 'Imdb User Rating', 'Imdbid', 'Metascore', 'Number Of Imdb User Votes'])
dataset_val = dataset_val.remove_columns(['Age Restrict', 'Akas', 'Awards', 'Budget', 'Cast', 'Casting Directors', 'Countries', 'Directors', 'Filename', 'Genres', 'Imdb Id', 'Keywords', 'Opening Weekend', 'Plot', 'Producers', 'Production Companies','Script Department', 'Synopsis', 'Taglines', 'Writers', 'Year', 'F1', 'Imdb User Rating', 'Imdbid', 'Metascore', 'Number Of Imdb User Votes'])

dataset_train = dataset_train.filter(lambda dataset: dataset["Plot Outline"]!=None)
dataset_train = dataset_train.filter(lambda dataset: dataset["Screenplay"]!=None)
dataset_train = dataset_train.filter(lambda x: len(str(x['Plot Outline'])) > 4)
dataset_train = dataset_train.filter(lambda x: len(str(x['Screenplay'])) > 4)


dataset_val = dataset_val.filter(lambda dataset: dataset_val["Plot Outline"]!=None)
dataset_val = dataset_val.filter(lambda dataset: dataset_val["Screenplay"]!=None)
dataset_val = dataset_val.filter(lambda x: len(str(x['Plot Outline'])) > 4)
dataset_val = dataset_val.filter(lambda x: len(str(x['Screenplay'])) > 4)

Filter:   0%|          | 0/2286 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1977 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1972 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1972 [00:00<?, ? examples/s]

Filter:   0%|          | 0/572 [00:00<?, ? examples/s]

Filter:   0%|          | 0/572 [00:00<?, ? examples/s]

Filter:   0%|          | 0/572 [00:00<?, ? examples/s]

Filter:   0%|          | 0/480 [00:00<?, ? examples/s]

# Testing BookSum Pre-Trained Model (Avatar)

In [11]:
import torch
torch.cuda.empty_cache()

dataset_avatar = dataset.filter(lambda dataset: dataset["Title"]=='Avatar')

hf_name = 'pszemraj/led-large-book-summary'

summarizer = pipeline(
    "summarization",
    hf_name,
    device=0 if torch.cuda.is_available() else -1,
)

# Limit input
wall_of_text = dataset_avatar['Screenplay'][0][:50000]

avatar_result = summarizer(
    wall_of_text,
    min_length=16,
    max_length=256, #Adjust if Cuda out of Memory
    no_repeat_ngram_size=3,
    encoder_no_repeat_ngram_size=3,
    repetition_penalty=3.5,
    num_beams=2, #Adjust if Cuda out of Memory
    early_stopping=True,
)

avatar_result

[{'summary_text': 'Jake, a 22-year-old combat veteran, is seen in a scene set in the city of Los Angeles. He is described as having a tattoo on his shoulder and being unable to walk. We are then shown Jake in his apartment where he is sitting in a chair pushed by a crowd of other soldiers. The bar where he works as a pool table keepers and chit chatters with others. They discuss the game they are playing, which is very difficult for him to understand since he has no feeling at all. Finally we see Jake in action as he drives a machine that is remotely controlled by a scientist. The scientist is Tommy who wants to learn more about the native people living in the jungle and their weapons. But Jake will not let him get away with just talking; instead, he needs to learn how to control the machine so that he can better understand the natural world around him. This is done through brute force. Next Jake goes out into the jungle to explore the local flora and fauna. He comes across a baby croc

In [25]:
import evaluate 
rouge = evaluate.load('rouge')

# booksum
print(rouge.compute(predictions=[avatar_result[0]['summary_text']], references=list(dataset_avatar['Plot Outline'])))

{'rouge1': 0.2702702702702703, 'rouge2': 0.02717391304347826, 'rougeL': 0.14594594594594595, 'rougeLsum': 0.14594594594594595}


# Base Model (~45 Mins for 100 Examples; 30 secs per example)

In [9]:
import evaluate 

dataset_base = dataset_train
dataset_base = dataset_base.shuffle(seed=42)
dataset_base = dataset_base.select(range(100))

# load tokenizer
tokenizer = LEDTokenizer.from_pretrained("pszemraj/led-base-book-summary")
model = LEDForConditionalGeneration.from_pretrained("pszemraj/led-base-book-summary").to("cuda").half()


def generate_answer(batch):
  inputs_dict = tokenizer(batch["Screenplay"], padding="max_length", max_length=16384, return_tensors="pt", truncation=True)
  input_ids = inputs_dict.input_ids.to("cuda")
  attention_mask = inputs_dict.attention_mask.to("cuda")
  global_attention_mask = torch.zeros_like(attention_mask)
  # put global attention on <s> token
  global_attention_mask[:, 0] = 1

  predicted_outline_ids = model.generate(input_ids, attention_mask=attention_mask, global_attention_mask=global_attention_mask, max_new_tokens=256)
  batch["predicted_plot"] = tokenizer.batch_decode(predicted_outline_ids, skip_special_tokens=True)
  return batch


base_result = dataset_base.map(generate_answer, batched=True, batch_size=2)

rouge = evaluate.load('rouge')

# booksum
print(rouge.compute(predictions=base_result['predicted_plot'], references=list(base_result['Plot Outline'])))


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

{'rouge1': 0.24477443314616837, 'rouge2': 0.02757124493270701, 'rougeL': 0.1277238582472771, 'rougeLsum': 0.12771104002910072}


# Large Model (~1Hr15Mins for 50 Examples; 90 secs per example)

In [3]:
import evaluate 

dataset_large = dataset_train
dataset_large = dataset_large.shuffle(seed=42)
dataset_large = dataset_large.select(range(50))

# load tokenizer
tokenizer = LEDTokenizer.from_pretrained("pszemraj/led-large-book-summary")
model = LEDForConditionalGeneration.from_pretrained("pszemraj/led-large-book-summary").to("cuda").half()


def generate_answer(batch):
  inputs_dict = tokenizer(batch["Screenplay"], padding="max_length", max_length=16384, return_tensors="pt", truncation=True)
  input_ids = inputs_dict.input_ids.to("cuda")
  attention_mask = inputs_dict.attention_mask.to("cuda")
  global_attention_mask = torch.zeros_like(attention_mask)
  # put global attention on <s> token
  global_attention_mask[:, 0] = 1

  predicted_outline_ids = model.generate(input_ids, attention_mask=attention_mask, global_attention_mask=global_attention_mask, max_new_tokens=256)
  batch["predicted_plot"] = tokenizer.batch_decode(predicted_outline_ids, skip_special_tokens=True)
  return batch


large_result = dataset_large.map(generate_answer, batched=True, batch_size=2)

rouge = evaluate.load('rouge')

# booksum
print(rouge.compute(predictions=large_result['predicted_plot'], references=list(large_result['Plot Outline'])))


Map:   0%|          | 0/50 [00:00<?, ? examples/s]

{'rouge1': 0.24497695728472, 'rouge2': 0.03260492483776796, 'rougeL': 0.1288160880504229, 'rougeLsum': 0.12880805708851784}


# Fine-tuned BookSum Model

## Training

In [ ]:
import torch
torch.cuda.empty_cache()

from datasets import load_dataset, load_metric
from transformers import (
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
)

# Train over 1K examples
ft_dataset_train = dataset_train.select(range(1000))
ft_dataset_val = dataset_val.select(range(50))

# load tokenizer
tokenizer = AutoTokenizer.from_pretrained("allenai/led-base-16384")


# max encoder set to max token length (could limit for efficiency purposes)
encoder_max_length = 16384
decoder_max_length = 256
batch_size = 1


def process_data_to_model_inputs(batch):
    # tokenize the inputs and labels
    inputs = tokenizer(
        batch["Screenplay"],
        padding="max_length",
        truncation=True,
        max_length=encoder_max_length,
    )
    outputs = tokenizer(
        batch["Plot Outline"],
        padding="max_length",
        truncation=True,
        max_length=decoder_max_length,
    )

    batch["input_ids"] = inputs.input_ids
    batch["attention_mask"] = inputs.attention_mask

    # create 0 global_attention_mask lists
    batch["global_attention_mask"] = len(batch["input_ids"]) * [
        [0 for _ in range(len(batch["input_ids"][0]))]
    ]

    # since above lists are references, the following line changes the 0 index for all samples
    batch["global_attention_mask"][0][0] = 1
    batch["labels"] = outputs.input_ids

    # We have to make sure that the PAD token is ignored
    batch["labels"] = [
        [-100 if token == tokenizer.pad_token_id else token for token in labels]
        for labels in batch["labels"]
    ]

    return batch


# map train data
ft_dataset_train = ft_dataset_train.map(
    process_data_to_model_inputs,
    batched=True,
    batch_size=batch_size,
    remove_columns=["Screenplay", "Plot Outline", "Title"],
)

# map val data
ft_dataset_val = ft_dataset_val.map(
    process_data_to_model_inputs,
    batched=True,
    batch_size=batch_size,
    remove_columns=["Screenplay", "Plot Outline", "Title"],
)

# set Python list to PyTorch tensor
ft_dataset_train.set_format(
    type="torch",
    columns=["input_ids", "attention_mask", "global_attention_mask", "labels"],
)

# set Python list to PyTorch tensor
ft_dataset_val.set_format(
    type="torch",
    columns=["input_ids", "attention_mask", "global_attention_mask", "labels"],
)

# enable fp16 apex training
training_args = Seq2SeqTrainingArguments(
    predict_with_generate=True,
    evaluation_strategy="steps",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    #fp16=True,
    #fp16_backend="apex",
    output_dir="./",
    logging_steps=250,
    eval_steps=5000,
    save_steps=500,
    warmup_steps=1500,
    save_total_limit=2,
    gradient_accumulation_steps=4,
)


# compute Rouge score during validation
def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = tokenizer.pad_token_id
    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

    rouge_output = rouge.compute(
        predictions=pred_str, references=label_str, rouge_types=["rouge2"]
    )["rouge2"].mid

    return {
        "rouge2_precision": round(rouge_output.precision, 4),
        "rouge2_recall": round(rouge_output.recall, 4),
        "rouge2_fmeasure": round(rouge_output.fmeasure, 4),
    }


# load model + enable gradient checkpointing & disable cache for checkpointing
led = AutoModelForSeq2SeqLM.from_pretrained("allenai/led-base-16384", gradient_checkpointing=True, use_cache=False)

# set generate hyperparameters
led.config.num_beams = 3
led.config.max_length = 256
led.config.min_length = 100
led.config.length_penalty = 2.0
led.config.early_stopping = True
led.config.no_repeat_ngram_size = 3


# instantiate trainer
trainer = Seq2SeqTrainer(
    model=led,
    tokenizer=tokenizer,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=ft_dataset_train,
    eval_dataset=ft_dataset_val,
)

# start training
trainer.train()

# save model 
trainer.save_model("booksum_model")
tokenizer.save_pretrained("booksum_tokenizer")

## Evaluate Model

In [ ]:
import torch
import evaluate
from datasets import load_dataset, load_metric
from transformers import LEDTokenizer, LEDForConditionalGeneration


dataset_test = dataset_val
dataset_test = dataset_test.shuffle(seed=42)
dataset_test = dataset_test.select(range(100))

dataset_test = dataset_test.filter(lambda dataset: dataset_test["Plot Outline"]!=None)
dataset_test = dataset_test.filter(lambda dataset: dataset_test["Screenplay"]!=None)
dataset_test = dataset_test.filter(lambda x: len(str(x['Plot Outline'])) > 4)
dataset_test = dataset_test.filter(lambda x: len(str(x['Screenplay'])) > 4)


# load tokenizer
tokenizer = LEDTokenizer.from_pretrained("booksum_model")
model = LEDForConditionalGeneration.from_pretrained("booksum_model").to("cuda").half()


def generate_answer(batch):
  inputs_dict = tokenizer(batch["Screenplay"], padding="max_length", max_length=8192, return_tensors="pt", truncation=True)
  input_ids = inputs_dict.input_ids.to("cuda")
  attention_mask = inputs_dict.attention_mask.to("cuda")
  global_attention_mask = torch.zeros_like(attention_mask)
  # put global attention on <s> token
  global_attention_mask[:, 0] = 1

  predicted_abstract_ids = model.generate(input_ids, attention_mask=attention_mask, global_attention_mask=global_attention_mask)
  batch["predicted_outline"] = tokenizer.batch_decode(predicted_abstract_ids, skip_special_tokens=True)
  return batch


ft_eval_result = dataset_test.map(generate_answer, batched=True, batch_size=4)

# load rouge

rouge = evaluate.load('rouge')

# booksum
print(rouge.compute(predictions=ft_eval_result['predicted_plot'], references=list(ft_eval_result['Plot Outline'])))